### Instalação de pacotes necessários

In [4]:
!pip install -U -q PyDrive
!pip install pyspellchecker
!pip install sentence-transformers
! pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 21.0 MB/s eta 0:00:00


### Import das biblioteca

In [5]:
from google.colab import files
import numpy as np
import pandas as pd
import nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import itertools
import os
os.environ["PYSPARK_SUBMIT_ARGS"] = "--driver-memory 8g pyspark-shell"
import io
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import nltk
import string
import unidecode
import joblib
from sentence_transformers import SentenceTransformer, util
from itertools import combinations
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Leitura dos arquivos

In [6]:
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['items_titles.csv']))
df_test = pd.read_csv(io.BytesIO(uploaded['items_titles_test.csv']))
df = df.drop_duplicates()
df_test = df_test.drop_duplicates()

Saving items_titles.csv to items_titles.csv
Saving items_titles_test.csv to items_titles_test.csv


### Avaliação da base

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 0 to 29999
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ITE_ITEM_TITLE  30000 non-null  object
dtypes: object(1)
memory usage: 468.8+ KB


In [16]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ITE_ITEM_TITLE  10000 non-null  object 
 1   SIMILARITY      10000 non-null  float64
 2   ITEM_MATCH      10000 non-null  object 
dtypes: float64(1), object(2)
memory usage: 234.5+ KB


In [17]:
df.describe()

,ITE_ITEM_TITLE
count,30000
unique,30000
top,Tênis Ascension Posh Masculino - Preto E Verme...
freq,1


In [18]:
df_test.describe()

,SIMILARITY
count,10000.000000
mean,0.017368
std,0.037409
min,0.000000
25%,0.000000
50%,0.000000
75%,0.015939
max,0.636437


In [19]:
df.head()

,ITE_ITEM_TITLE
0,Tênis Ascension Posh Masculino - Preto E Verme...
1,Tenis Para Caminhada Super Levinho Spider Corr...
2,Tênis Feminino Le Parc Hocks Black/ice Origina...
3,Tênis Olympikus Esportivo Academia Nova Tendên...
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...


In [20]:
df_test.head()

,ITE_ITEM_TITLE,SIMILARITY,ITEM_MATCH
0,Tênis Olympikus Esporte Valente - Masculino Kids,0.046985,Tênis Infantil Olympikus Valente Kids Masculino
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...,0.000000,Bicicleta Barra Forte Aro 24
2,Tênis Usthemp Slip-on Temático - Labrador 2,0.008585,Tênis Usthemp Slip-on Temático - Lunna Labrad...
3,Tênis Casual Feminino Moleca Tecido Tie Dye,0.013102,Tênis Tie Dye Feminino Casual
4,Tênis Star Baby Sapatinho Conforto + Brinde,0.000000,Tênis Infantil Ortopé Sport Baby Sapatinho Beb...


### Pré Processamento

In [ ]:
df_test = df_test.rename(columns={"ITE_ITEM_TITLE": "ITE_ITEM_TITLE_TEST"})

In [ ]:
df = df.drop_duplicates()
df_test = df_test.drop_duplicates()

In [ ]:
stop_words = set(stopwords.words('portuguese'))
df['ITE_ITEM_TITLE'] = df['ITE_ITEM_TITLE'].apply(lambda x: ' '.join([word.lower() for word in word_tokenize(x) if word.lower() not in stop_words]))
df_test['ITE_ITEM_TITLE'] = df_test['ITE_ITEM_TITLE'].apply(lambda x: ' '.join([word.lower() for word in word_tokenize(x) if word.lower() not in stop_words]))

In [ ]:
nltk.download('punkt')

def remove_special_chars(text):
    """
    Remove caracteres especiais de um texto utilizando NLP.
    """
    # Tokeniza o texto em palavras
    words = nltk.word_tokenize(text)
    
    # Remove os caracteres especiais de cada palavra
    words_clean = [word for word in words if word.isalnum()]
    
    # Junta as palavras novamente em um texto
    text_clean = ' '.join(words_clean)
    
    return text_clean

df['ITE_ITEM_TITLE'] = df['ITE_ITEM_TITLE'].apply(remove_special_chars)
df_test['ITE_ITEM_TITLE'] = df_test['ITE_ITEM_TITLE'].apply(remove_special_chars)

# definir a função que remove acentos de um texto
def remover_acentos(text):
    return unidecode.unidecode(text)

# aplicar a função para remover acentos na coluna 'texto' do dataframe
df['ITE_ITEM_TITLE'] = df['ITE_ITEM_TITLE'].apply(remover_acentos)
df_test['ITE_ITEM_TITLE'] = df_test['ITE_ITEM_TITLE'].apply(remover_acentos)

### Modelagem utilizando como base de treino items_titles.csv

In [11]:
# Leitura dos dados de treino e teste
df_train = pd.read_csv('items_titles.csv')
df_test = pd.read_csv('items_titles_test.csv')

# Combinando descrições para a base de teste
test_items = df_test['ITE_ITEM_TITLE'].values.tolist()

# Criando um vetorizador TF-IDF
vectorizer = TfidfVectorizer()

# Fitando o vetorizador com os dados de treino
vectorizer.fit(df_train['ITE_ITEM_TITLE'].values)

# Codificando todos os textos de uma vez para vetores com o modelo TF-IDF
vectors_train = vectorizer.transform(df_train['ITE_ITEM_TITLE'].values)
vectors_test = vectorizer.transform(test_items)

# Calculando a matriz de similaridade entre as descrições de treino e teste
similarity_matrix = cosine_similarity(vectors_test, vectors_train)

# Pegando as similaridades apenas das combinações (i, j) onde i == j
# já que a similaridade da descrição i com ela mesma é sempre 1
similarities = similarity_matrix.diagonal()

# Adicionando coluna com similaridade no dataframe de teste
df_test['SIMILARITY'] = similarities

# Adicionando coluna com o item correspondente
df_test['ITEM_MATCH'] = df_train.iloc[similarity_matrix.argmax(axis=1)]['ITE_ITEM_TITLE'].values

In [14]:
df_test.sort_values(by=['SIMILARITY'], ascending=False)

,ITE_ITEM_TITLE,SIMILARITY,ITEM_MATCH
6227,Kit 2 Pares Sapatenis Tênis,0.636437,Kit 2 Pares Sapatenis Tênis
21,Tenis Feminino Academia Caminhada Crossfit Cor...,0.459619,Tenis Feminino Calce Caminhada Crossfit Corri...
6128,Tenis Mizuno Wave Prophecy 7 Camaleão,0.440869,Tênis Mizuno Wave Prophecy 6 Camaleão Envio Im...
77,Vapor Max Plus,0.434716,Vapor Max Plus
2859,Slip On Preto Plataforma Via Marte - Tenis Fem...,0.394782,Tênis Feminino Via Marte Flatform Slip On Plat...
...,...,...,...
6019,Sapatênis Masculino Couro Preto Montreal Ferra...,0.000000,Sapatênis Masculino Couro Preto Montreal Ferra...
6017,Bicicleta Retrô Aro 26 Com Cesta E Bagageiro V...,0.000000,Bicicleta Aro 26 Retrô Vintage Cesta E Bagagei...
6016,Tênis Ramarim 21-80203,0.000000,Tênis Feminino Esportivo Ramarim 21-80203
3191,Sapatênis Masculino Ferracini Casual Sport Em ...,0.000000,Sapatênis Masculino Casual Ferracini Em Couro


### Modelagem - Aplicando Transfer Learning utilizando um modelo treinado com uma base muito maior

In [ ]:
# Carregando o modelo BERT pré-treinado
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

# Leitura dos dados de treino e teste
df_train = df
test_items = df_test['ITE_ITEM_TITLE'].values.tolist()

# Ajustando o tamanho do chunk para o número de cores disponíveis
batch_size = 1000

# Iterando sobre os lotes de test_items para codificá-los com o modelo BERT
vectors = []
for i in tqdm(range(0, len(test_items), batch_size)):
    batch_items = test_items[i:i+batch_size]
    batch_vectors = model.encode(batch_items)
    vectors.append(batch_vectors)

# Concatenando as matrizes de vetores em um único array
vectors = np.concatenate(vectors)

# Calculando a matriz de distâncias
distances = cosine_similarity(vectors)

# Obtendo a similaridade para cada pedaço de combinações de forma paralelizada
similarities = []
n_items = len(test_items)
n_combinations = (n_items * (n_items - 1)) // 2
chunk_combinations = list(combinations(range(n_items), 2))
chunk_size = batch_size * 2
n_chunks = int(np.ceil(n_combinations / chunk_size))
for i in tqdm(range(n_chunks)):
    start = i * chunk_size
    end = min(start + chunk_size, n_combinations)
    chunk_indices = chunk_combinations[start:end]
    chunk_vectors = vectors[np.array(chunk_indices).flatten()]
    chunk_distances = distances[np.array(chunk_indices)[:, 0], np.array(chunk_indices)[:, 1]]
    chunk_similarity = np.asarray(chunk_distances)
    chunk_similarity = (chunk_similarity + 1) / 2
    similarities.append(chunk_similarity.tolist())

# Concatenando as similaridades
similarities = np.concatenate(similarities)

# Adicionando colunas com similaridade nos dataframes de treino e teste
df_combinations = pd.DataFrame(list(combinations(test_items, 2)), columns=['ITEM_TITLE_1', 'ITEM_TITLE_2'])
df_combinations['SIMILARITY'] = similarities

df_combinations['SIMILARITY'] = df_combinations['SIMILARITY'].astype(float)
#df_combinations['SIMILARITY'] = (df_combinations['SIMILARITY'] / 100)


100%|██████████| 24998/24998 [02:22<00:00, 175.02it/s]


In [ ]:
df_combinations.sort_values(by=['SIMILARITY'], ascending=False).head(10)

,ITEM_TITLE_1,ITEM_TITLE_2,SIMILARITY
9616510,Tv Portátil Tomate Mtm-808 Led 8 Polegadas 10...,Tv Portátil Tomate Mtm-808 Led 8 Polegadas 100...,1.000000
4632483,Tenis D&g Milano,Tenis D & G Milano,1.000000
27952511,Bicicleta Feminin Infantil Nathor Top Girls A...,Bicicleta Feminin Infantil Nathor Top Girls A...,1.000000
15618237,Tênis Slip On,Tênis Slip On,1.000000
7853855,Mountain Bike 29 Dropp 27v Freio Hidraulico,Mountain Bike 29 Dropp 27v Freio Hidraulico,1.000000
25896194,Sapatênis Tênis Masculino Couro Legítimo Cano ...,Sapatênis Tênis Masculino Couro Legítimo Cano...,1.000000
5352486,Tênis Converse All Star Cano Alto,Tênis Converse All Star Cano Alto,1.000000
27011907,Tenis De Basquete,Tênis De Basquete,1.000000
37336476,Sapatenis Masculino Da Polo Lançamento 30% Off,Sapatenis Masculino Da Polo Lançamento 30% Off,1.000000
26699685,Tênis Menino Infantil Bibi Roller 2.0 Prático ...,Tênis Infantil Menino Bibi Roller 2.0 Prático ...,0.999103


In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# Carregando o modelo BERT pré-treinado
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

# Leitura dos dados de treino e teste
df_train = df
test_items = df_test['ITE_ITEM_TITLE'].values.tolist()

# Ajustando o tamanho do chunk para o número de cores disponíveis
chunk_size = 1000
n_cores = joblib.cpu_count()
chunk_size = chunk_size * n_cores

# Codificando todos os textos de uma vez para vetores com o modelo BERT
vectors = model.encode(test_items)

# Calculando a matriz de distâncias
distances = cosine_similarity(vectors)

# Obtendo a similaridade para cada pedaço de combinações de forma paralelizada
similarities = []
n_items = len(test_items)
n_combinations = (n_items * (n_items - 1)) // 2
chunk_combinations = list(combinations(range(n_items), 2))
n_chunks = int(np.ceil(n_combinations / chunk_size))
for i in tqdm(range(n_chunks)):
    start = i * chunk_size
    end = min(start + chunk_size, n_combinations)
    chunk_indices = chunk_combinations[start:end]
    chunk_vectors = vectors[np.array(chunk_indices).flatten()]
    chunk_distances = distances[np.array(chunk_indices)[:, 0], np.array(chunk_indices)[:, 1]]
    chunk_similarity = np.asarray(chunk_distances)
    chunk_similarity = (chunk_similarity + 1) / 2
    similarities.append(chunk_similarity.tolist())

# Concatenando as similaridades
similarities = np.concatenate(similarities)

# Adicionando colunas com similaridade nos dataframes de treino e teste
df_combinations = pd.DataFrame(list(combinations(test_items, 2)), columns=['ITEM_TITLE_1', 'ITEM_TITLE_2'])
df_combinations['SIMILARITY'] = similarities

df_combinations['SIMILARITY'] = df_combinations['SIMILARITY'].astype(float)
#df_combinations['SIMILARITY'] = (df_combinations['SIMILARITY'] / 100)


In [ ]:
df_combinations['SIMILARITY'] = df_combinations['SIMILARITY'] * 100

In [ ]:
df_combinations.sort_values(by=['SIMILARITY'], ascending=False)

,ITEM_TITLE_1,ITEM_TITLE_2,SIMILARITY
147868,Tênis Fila Racer For All Masculino Corrida - C...,Tênis Fila Racer For All Feminino Corrida - Ca...,0.989346
141866,Sapatênis Tenis Confort Couro Legítimo Feminin...,Tenis Feminino Sapatenis Confortavel Couro Leg...,0.987219
112047,Bicicleta Para Menina Branca E Rosa Aro 20 Ult...,Bicicleta Infantil Menino Aro 20 Bicolor Ultra...,0.985533
287199,Bicicleta Aro 20 Preta Kit Aero Laranja C Capa...,Bicicleta Aro 20 Preta Kit Aero Preto C/ Acele...,0.983308
421793,Sapatênis Cano Alto Veloster Abotinado Ranster...,Sapatênis Cano Alto Masculino Ranster Ref; 1006,0.981443
...,...,...,...
418188,New Balance Running Course,Smart Tv Samsung 48'Un48j5200agxzd Display Com...,0.537808
418956,Tênis Converse Chuck Taylor All Star Seasonal ...,Sagitta Se,0.531970
440711,Sagitta Se,Sapatilha Mattos Racing Ttr Mtb Bike Ciclismo ...,0.522929
319456,Sapatilha Scott Mtb Team Boa 2020 Original Mo...,Sagitta Se,0.519398


### Análise/Oportunidades

1. Vale destacar que quando treinado o modelo com a base do mercado livre a performance não foi tão robusta, classificando itens quase idênticos com valores baixos ou até zero, contudo quando utilizado técnica de transfer learning e buscando um modelo pré treinando numa base bem maior temmos valores de similaridade que fazem bem mais sentido
2. Como oportunidade novamente se tivermos mais poder de computação como uma super máquina com mais cores podemos realizar computação paralela e deixar um cenário bem mais robusto rodando o que tende a melhor muito a perfomance
3. Utilizei estratégias como computação paralela e divisão em batchs e chunks visando otimizar o código rodar
4. No pré processamento utilizei tratamentos necessários para se trabalhar com dados textuais visando facilitar o calculo de similaridade pelo modelo